In [458]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import RandomOverSampler



In [459]:
pd.set_option('display.max_columns', None)

In [460]:
df_01 = pd.read_csv("../Resources/attributesBillboard.csv")
print(df_01.info())
df_01.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4774 entries, 0 to 4773
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        4774 non-null   int64  
 1   Artist            4774 non-null   object 
 2   Name              4774 non-null   object 
 3   Weekly.rank       4774 non-null   int64  
 4   Peak.position     4229 non-null   float64
 5   Weeks.on.chart    4229 non-null   float64
 6   Week              4774 non-null   object 
 7   Date              3938 non-null   object 
 8   Genre             4774 non-null   object 
 9   Writing.Credits   4279 non-null   object 
 10  Lyrics            4774 non-null   object 
 11  Features          717 non-null    object 
 12  Acousticness      4774 non-null   float64
 13  Album             4774 non-null   object 
 14  Danceability      4774 non-null   float64
 15  Duration          4774 non-null   int64  
 16  Energy            4774 non-null   float64


,Unnamed: 0,Artist,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features,Acousticness,Album,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Popularity,Speechiness,Tempo,TimeSignature,Valence
0,0,jonas brothers,sucker,6,1.0,17.0,2019-07-06,"March 1, 2019","Alternative Pop,Boy Band,Teen Pop,Pop-Rock,Pop","Ryan tedder, Louis bell, Frank dukes, Nick jon...",Sucker \nWe go together\nBetter than birds of ...,NaN,0.04270,Happiness Begins,0.842,181027,0.734,False,0.0,0.1060,-5.065,0,81,0.0588,137.958,4,0.952
1,1,taylor swift,you need to calm down,13,2.0,2.0,2019-07-06,"June 14, 2019","Synth-Pop,LGBTQ+,Pop","Joel little, Taylor swift",You Need To Calm Down \nYou are somebody that ...,NaN,0.00929,Lover,0.771,171360,0.671,False,0.0,0.0637,-5.617,1,89,0.0553,85.026,4,0.714
2,2,panic! at the disco,"hey look ma, i made it",24,24.0,11.0,2019-07-06,"June 22, 2018","Pop-Rock,Jazz Fusion,Alternative,Alternative P...","Jake sinclair, Michael angelakos, Dillon franc...","Hey Look Ma, I Made It \nAll my life, been hus...",NaN,0.01370,Pray for the Wicked,0.577,169667,0.833,False,0.0,0.1210,-3.337,1,81,0.0695,107.936,4,0.580
3,3,lee brice,rumor,26,25.0,16.0,2019-07-06,"November 3, 2017",Country,"Kyle jacobs, Ashley gorley, Lee brice",Rumor \nGirl you know I've known you forever\n...,NaN,0.74900,Lee Brice,0.655,198440,0.560,False,0.0,0.1150,-6.857,1,79,0.0486,140.975,4,0.599
4,4,panic! at the disco,high hopes,32,4.0,47.0,2019-07-06,"May 23, 2018","Adult Alternative,Rock,Power Pop,Pop-Rock,Alte...","Sam hollander, Cook classics, Tayla parx, Jake...","High Hopes \nHigh, high hopes\nHad to have hig...",NaN,0.19300,Pray for the Wicked,0.579,190947,0.904,False,0.0,0.0640,-2.729,1,87,0.0618,82.014,4,0.681


In [461]:
df_01.columns

Index(['Unnamed: 0', 'Artist', 'Name', 'Weekly.rank', 'Peak.position',
       'Weeks.on.chart', 'Week', 'Date', 'Genre', 'Writing.Credits', 'Lyrics',
       'Features', 'Acousticness', 'Album', 'Danceability', 'Duration',
       'Energy', 'Explicit', 'Instrumentalness', 'Liveness', 'Loudness',
       'Mode', 'Popularity', 'Speechiness', 'Tempo', 'TimeSignature',
       'Valence'],
      dtype='object')

In [462]:
df_01 = df_01.drop(columns= ["Unnamed: 0", "Artist","Name","Genre","Writing.Credits","Lyrics","Album","Week","Date"])
df_01['Features'] = df_01['Features'].notna().astype(int)
df_01.head()

,Weekly.rank,Peak.position,Weeks.on.chart,Features,Acousticness,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Popularity,Speechiness,Tempo,TimeSignature,Valence
0,6,1.0,17.0,0,0.04270,0.842,181027,0.734,False,0.0,0.1060,-5.065,0,81,0.0588,137.958,4,0.952
1,13,2.0,2.0,0,0.00929,0.771,171360,0.671,False,0.0,0.0637,-5.617,1,89,0.0553,85.026,4,0.714
2,24,24.0,11.0,0,0.01370,0.577,169667,0.833,False,0.0,0.1210,-3.337,1,81,0.0695,107.936,4,0.580
3,26,25.0,16.0,0,0.74900,0.655,198440,0.560,False,0.0,0.1150,-6.857,1,79,0.0486,140.975,4,0.599
4,32,4.0,47.0,0,0.19300,0.579,190947,0.904,False,0.0,0.0640,-2.729,1,87,0.0618,82.014,4,0.681


In [463]:
df_01_dummies = pd.get_dummies(df_01)
df_01_dummies.head()

,Weekly.rank,Peak.position,Weeks.on.chart,Features,Acousticness,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Popularity,Speechiness,Tempo,TimeSignature,Valence
0,6,1.0,17.0,0,0.04270,0.842,181027,0.734,False,0.0,0.1060,-5.065,0,81,0.0588,137.958,4,0.952
1,13,2.0,2.0,0,0.00929,0.771,171360,0.671,False,0.0,0.0637,-5.617,1,89,0.0553,85.026,4,0.714
2,24,24.0,11.0,0,0.01370,0.577,169667,0.833,False,0.0,0.1210,-3.337,1,81,0.0695,107.936,4,0.580
3,26,25.0,16.0,0,0.74900,0.655,198440,0.560,False,0.0,0.1150,-6.857,1,79,0.0486,140.975,4,0.599
4,32,4.0,47.0,0,0.19300,0.579,190947,0.904,False,0.0,0.0640,-2.729,1,87,0.0618,82.014,4,0.681


In [464]:
df_01_dummies["Popularity"].describe()

count    4774.000000
mean       39.847926
std        21.823721
min         0.000000
25%        23.000000
50%        42.000000
75%        58.000000
max        89.000000
Name: Popularity, dtype: float64

In [465]:
df_01_dummies = df_01_dummies.dropna()
df_01_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4229 entries, 0 to 4773
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Weekly.rank       4229 non-null   int64  
 1   Peak.position     4229 non-null   float64
 2   Weeks.on.chart    4229 non-null   float64
 3   Features          4229 non-null   int32  
 4   Acousticness      4229 non-null   float64
 5   Danceability      4229 non-null   float64
 6   Duration          4229 non-null   int64  
 7   Energy            4229 non-null   float64
 8   Explicit          4229 non-null   bool   
 9   Instrumentalness  4229 non-null   float64
 10  Liveness          4229 non-null   float64
 11  Loudness          4229 non-null   float64
 12  Mode              4229 non-null   int64  
 13  Popularity        4229 non-null   int64  
 14  Speechiness       4229 non-null   float64
 15  Tempo             4229 non-null   float64
 16  TimeSignature     4229 non-null   int64  
 17  

In [466]:
df_01_dummies["is_popular"] = df_01_dummies["Popularity"] >= 58
df_01_dummies = df_01_dummies.drop(columns=["Popularity"])
df_01_dummies.head()

,Weekly.rank,Peak.position,Weeks.on.chart,Features,Acousticness,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Speechiness,Tempo,TimeSignature,Valence,is_popular
0,6,1.0,17.0,0,0.04270,0.842,181027,0.734,False,0.0,0.1060,-5.065,0,0.0588,137.958,4,0.952,True
1,13,2.0,2.0,0,0.00929,0.771,171360,0.671,False,0.0,0.0637,-5.617,1,0.0553,85.026,4,0.714,True
2,24,24.0,11.0,0,0.01370,0.577,169667,0.833,False,0.0,0.1210,-3.337,1,0.0695,107.936,4,0.580,True
3,26,25.0,16.0,0,0.74900,0.655,198440,0.560,False,0.0,0.1150,-6.857,1,0.0486,140.975,4,0.599,True
4,32,4.0,47.0,0,0.19300,0.579,190947,0.904,False,0.0,0.0640,-2.729,1,0.0618,82.014,4,0.681,True


In [467]:
df_01_dummies["is_popular"].value_counts()

is_popular
False    3125
True     1104
Name: count, dtype: int64

In [468]:
# Split our preprocessed data into our features and target arrays
X = df_01_dummies.drop(columns = ["is_popular"])
y = df_01_dummies["is_popular"]
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [469]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Neural Network

In [470]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_features = len(X_train_scaled[0])
num_neurons_hl1 = 30
num_neurons_hl2 = 30
num_neurons_hl3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = num_neurons_hl1, input_dim = num_input_features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = num_neurons_hl2, activation = "relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = num_neurons_hl3, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_51 (Dense)            (None, 30)                540       
                                                                 
 dense_52 (Dense)            (None, 30)                930       
                                                                 
 dense_53 (Dense)            (None, 30)                930       
                                                                 
 dense_54 (Dense)            (None, 1)                 31        
                                                                 
Total params: 2431 (9.50 KB)
Trainable params: 2431 (9.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [471]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])


In [472]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 50)


Epoch 1/50


100/100 [==============================] - 2s 4ms/step - loss: 0.5852 - accuracy: 0.7405
Epoch 2/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5493 - accuracy: 0.7433
Epoch 3/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5416 - accuracy: 0.7439
Epoch 4/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5355 - accuracy: 0.7455
Epoch 5/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5315 - accuracy: 0.7461
Epoch 6/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5264 - accuracy: 0.7521
Epoch 7/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5222 - accuracy: 0.7512
Epoch 8/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5179 - accuracy: 0.7528
Epoch 9/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5159 - accuracy: 0.7562
Epoch 10/50
100/100 [==============================] - 0s 2ms/step - loss: 0.5093 - accuracy: 0.7629
Epoch 11/5

In [473]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

34/34 - 0s - loss: 0.7298 - accuracy: 0.7051 - 179ms/epoch - 5ms/step
Loss: 0.7297839522361755, Accuracy: 0.7051039934158325


## Logistic Regression

In [474]:
# Instantiate the Logistic Regression model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data
logistic_regression_model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [475]:
predictions = logistic_regression_model.predict(X_test)
predictions

array([False, False, False, ..., False, False, False])

In [476]:
# Print the balanced_accuracy score of the model
print(f"Balanced Accuracy Score: {balanced_accuracy_score(y_test,predictions)} %")

Balanced Accuracy Score: 0.49934980494148246 %


In [477]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
print(cm)

[[768   1]
 [289   0]]


In [478]:
# Print the classification report for the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

       False       0.73      1.00      0.84       769
        True       0.00      0.00      0.00       289

    accuracy                           0.73      1058
   macro avg       0.36      0.50      0.42      1058
weighted avg       0.53      0.73      0.61      1058



## Logistic Regression Resampled

In [479]:
# Instantiate the random oversampler model
random_over_sampler_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_res, y_res = random_over_sampler_model.fit_resample(X_train, y_train)

In [480]:
# Count the distinct values of the resampled labels data
print(y_res.value_counts())

is_popular
True     2356
False    2356
Name: count, dtype: int64


In [481]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model_resampled = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
logistic_regression_model_resampled.fit(X_res, y_res)

# Make a prediction using the testing data
predictions_resampled = logistic_regression_model_resampled.predict(X_test)

In [482]:
# Print the balanced_accuracy score of the model 
print(f"Balanced Accuracy Score Resampled: {balanced_accuracy_score(y_test,predictions_resampled)} %")

Balanced Accuracy Score Resampled: 0.5 %


In [483]:
# Generate a confusion matrix for the model
cm_resampled = confusion_matrix(y_test, predictions_resampled)
print(cm_resampled)

[[769   0]
 [289   0]]


In [484]:
# Print the classification report for the model
print(classification_report(y_test, predictions_resampled))

              precision    recall  f1-score   support

       False       0.73      1.00      0.84       769
        True       0.00      0.00      0.00       289

    accuracy                           0.73      1058
   macro avg       0.36      0.50      0.42      1058
weighted avg       0.53      0.73      0.61      1058



c:\Users\Aspire-PC\anaconda3\envs\orange3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aspire-PC\anaconda3\envs\orange3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aspire-PC\anaconda3\envs\orange3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
